# Yale Face Dataset B

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.constrained_layout.use'] = True

mat_contents = scipy.io.loadmat('allFaces.mat')
faces = mat_contents['faces']
m = int(mat_contents['m'])
n = int(mat_contents['n'])
nfaces = np.ndarray.flatten(mat_contents['nfaces'])

y = np.zeros((faces.shape[1],))
j = 0
classes = list(range(len(nfaces)))
for i in nfaces:
  y[j:j+i] = classes.pop(0)
  j = j + i

print("Total dataset size:")
print(f"n_samples: {faces.shape[1]}")
print(f"n_features: {m*n}")
print(f"n_classes: {len(nfaces)}")

In [ ]:
allPersons = np.zeros((n*6,m*6))
count = 0

for j in range(6):
    for k in range(6):
        allPersons[j*n : (j+1)*n, k*m : (k+1)*m] = np.reshape(faces[:,np.sum(nfaces[:count])],(m,n)).T
        count += 1

img = plt.imshow(allPersons)
img.set_cmap('gray')
plt.axis('off')
plt.show()

In [ ]:
for person in range(len(nfaces)):
    subset = faces[:,sum(nfaces[:person]) : sum(nfaces[:(person+1)])]
    allFaces = np.zeros((n*8,m*8))

    count = 0

    for j in range(8):
        for k in range(8):
            if count < nfaces[person]:
                allFaces[j*n:(j+1)*n,k*m:(k+1)*m] = np.reshape(subset[:,count],(m,n)).T
                count += 1

    img = plt.imshow(allFaces)
    img.set_cmap('gray')
    plt.axis('off')

    fileName = f"P{person+1:02d}"
    plt.show()

In [ ]:
sns.set(style="darkgrid")
plt.rc('xtick', labelsize=22)
plt.rc('ytick', labelsize=22)
plt.rc('axes', labelsize=22)

fig = plt.figure(figsize=(18, 10))
ax = sns.countplot(x=y, hue=y, palette='icefire', legend=False)

ax.set_xticks(range(len(set(y))))
ax.set_xticklabels(range(1, 39))

ax.set_ylabel("Count")
ax.set_xlabel("Person")
plt.show()

# Eigenfaces with PCA

In [ ]:
from time import time
import os

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
trainingFaces = faces[:,:np.sum(nfaces[:36])]
avgFace = np.mean(trainingFaces,axis=1)

# Compute eigenfaces on mean-subtracted training data
print("Computing eigenfaces / Performing SVD on face library")
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
t0 = time()
U, S, VT = np.linalg.svd(X,full_matrices=0)
print(f"done in {time()-t0:0.3f}s")

In [ ]:
# Plot avg face
fig1 = plt.figure()
ax = fig1.add_subplot()
img_avg = ax.imshow(np.reshape(avgFace,(m,n)).T)
img_avg.set_cmap('gray')
plt.axis('off')
plt.title("Average face")
plt.show()

# Plot first 5 eigenfaces
fig2 = plt.figure(figsize=(10, 3))
ax = fig2.add_subplot(161)
img_avg = ax.imshow(np.reshape(avgFace,(m,n)).T)
img_avg.set_cmap('gray')
ax.set_title("Average Face")
plt.axis('off')

for i in range(5):
  ax = fig2.add_subplot(1, 6, i+2)
  img = ax.imshow(np.reshape(U[:,i],(m,n)).T)
  img.set_cmap('gray')
  ax.set_title(f"Eigenface {i+1}")
  plt.axis('off')

plt.show()

In [ ]:
fig3 = plt.figure(figsize=(10, 5), tight_layout=True)
ax1 = fig3.add_subplot(121)
ax1.semilogy(S,'-o',color='k')
ax1.set(xlabel='r', ylabel='Singular values $\sigma{}_r$')
ax2 = fig3.add_subplot(122)
ax2.plot(np.cumsum(S)/np.sum(S),'-o',color='k')
ax2.set(xlabel='r', ylabel='Cumulative energy')
plt.show()

In [ ]:
n_components = 150
var = np.sum(S[:n_components])/np.sum(S)
print(f"Amount of variance captured by first {n_components} eigenfaces: {var*100:.1f}")

In [ ]:
testFace = faces[:,np.sum(nfaces[:36])]

testFaceMS = testFace - avgFace
r_list = [25, 50, 100, 200, 400, 800, 1600]
width = 4
height = int(np.ceil(len(r_list)/width))
fig4 = plt.figure(figsize=(12, 8))
ax = fig4.add_subplot(height, width, 1)
img = ax.imshow(np.reshape(testFace,(m,n)).T)
img.set_cmap('gray')
ax.set_title('Original Image')
plt.axis('off')

for r in r_list:
    reconFace = avgFace + U[:,:r] @ (U[:,:r].T @ testFaceMS)
    ax = fig4.add_subplot(height, width, r_list.index(r)+2)
    img = ax.imshow(np.reshape(reconFace,(m,n)).T)
    img.set_cmap('gray')
    ax.set_title('r = ' + str(r))
    plt.axis('off')

plt.show()

In [ ]:
testFace = faces[:,np.sum(nfaces[:35])]

testFaceMS = testFace - avgFace
r_list = [25, 50, 100, 200, 400, 800, 1600]
plt.imshow(np.reshape(testFace,(m,n)).T)
plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

print(f"Original size: {testFace.shape}")

for r in r_list:
    comprFace = U[:,:r].T @ testFaceMS
    print(f"Compressed size: {comprFace.shape}")

In [ ]:
P1num = 2
P2num = 7

P1 = faces[:, np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num])]
P2 = faces[:, np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num])]

P1 = P1 - np.tile(avgFace, (P1.shape[1],1)).T
P2 = P2 - np.tile(avgFace, (P2.shape[1],1)).T

PCAmodes = [5, 6]

PCACoordsP1 = U[:, PCAmodes-np.ones_like(PCAmodes)].T @ P1
PCACoordsP2 = U[:, PCAmodes-np.ones_like(PCAmodes)].T @ P2

fig, ax = plt.subplots()

ax.plot(PCACoordsP1[0, :], PCACoordsP1[1, :], 'd', color='k', label=f"Person {P1num}")
ax.plot(PCACoordsP2[0, :], PCACoordsP2[1, :], '^', color='r', label=f"Person {P2num}")

ax.set(xlabel="PC 5", ylabel="PC 6")
ax.grid()
ax.legend()
plt.show()

# PCA Feature Extraction

In [ ]:
from utils import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams.update({'font.size': 18})

X = faces.T
y = np.zeros((faces.shape[1], 1))

j = 0
classes = list(range(len(nfaces)))
for i in nfaces:
  y[j:j+i] = classes.pop(0)
  j = j + i

In [ ]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=y)

In [ ]:
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_tv.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_tv)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, m, n))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_tv_pca = pca.transform(X_tv)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

In [ ]:
person = 1
subset = faces[:,sum(nfaces[:person]) : sum(nfaces[:(person+1)])]
subset = subset[:, :8] # only take first 8 faces of person 1
data = pca.transform(subset.T)
fileName = "PCAfeatures"

plot_features(data[:, :16])

In [ ]:
# Plot first 5 eigenfaces
fig1 = plt.figure(figsize=(10, 3))

for i in range(5):
  ax = fig1.add_subplot(1, 5, i+1)
  img = ax.imshow(eigenfaces[i].T)
  img.set_cmap('gray')
  ax.set_title(f"Eigenface {i+1}")
  plt.axis('off')

plt.show()

In [ ]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_tv_pca, y_tv.ravel())
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

target_names = [f"Person{n}" for n in range(1,39)]

print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))
acc_pca = accuracy_score(y_test, y_pred)
print(f"PCA Baseline Accuracy: {acc_pca:.2%}")

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

plt.rcParams.update({'font.size': 10})
plt.rcParams['figure.figsize'] = [12, 12]

disp = ConfusionMatrixDisplay.from_estimator(
    clf,
    X_test_pca,
    y_test,
    cmap=plt.cm.Blues,
    display_labels=range(1, 39)
)

plt.grid(False)
plt.show()

In [ ]:
# plot the result of the prediction on a portion of the test set

prediction_titles = [title(y_pred, y_test, target_names, i) for i in range(y_pred.shape[0])]
plot_gallery(X_test, prediction_titles, m, n)

# Autoencoder

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, LocallyConnected2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

plt.rcParams.update({'font.size': 11})
plt.rcParams['figure.constrained_layout.use'] = True

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))

In [ ]:
allFaces = faces.T
y = np.zeros((faces.shape[1], 1))
M = allFaces.reshape(allFaces.shape[0], m, n)
dataset = preprocess_faces(M)

j = 0
classes = list(range(len(nfaces)))
for i in nfaces:
  y[j:j+i] = classes.pop(0)
  j = j + i

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))
model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))
model.add(Convolution2D(16, (9, 9), activation='relu', name='C3'))
model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))
model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )
model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))
model.add(Flatten(name='F0'))
model.add(Dense(4096, activation='relu', name='F7'))
model.add(Dropout(rate=0.5, name='D0'))
model.add(Dense(8631, activation='softmax', name='F8'))

model.summary()

encoder = Model(inputs=model.layers[0].input, outputs=model.layers[-3].output)

In [ ]:
weights = get_weights()
model.load_weights(weights)

In [ ]:
print("Feature generation of face library")
t0 = time()
person = 0
subset = dataset[sum(nfaces[:person]): sum(nfaces[:(person+1)])]
subset = encoder.predict(subset/255)
print("done in %0.3fs" % (time() - t0))
plot_features(subset[:8, :16])

In [ ]:
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [ ]:
img1_path = 1
img2_path = 2
target = True

print(img1_path, " and ", img2_path)

img1 = dataset[img1_path]/255
img2 = dataset[img2_path]/255

fig = plt.figure()
fig.add_subplot(1,2, 1)
plt.imshow(img1)
plt.xticks([]); plt.yticks([])
fig.add_subplot(1,2, 2)
plt.imshow(img2)
plt.xticks([]); plt.yticks([])
plt.show(block=True)

img1_embedding = encoder.predict(np.expand_dims(img1, axis=0))
img2_embedding = encoder.predict(np.expand_dims(img2, axis=0))


euclidean_distance = findEuclideanDistance(img1_embedding, img2_embedding)
euclidean_l2_distance = findEuclideanDistance(l2_normalize(img1_embedding), l2_normalize(img2_embedding))

print("Euclidean L2 distance: ", euclidean_l2_distance)
print("Actual: ", target, end = '')

if euclidean_l2_distance <= 0.55:
    verified = True
else:
    verified = False

print(" - Predicted: ", verified)

In [ ]:
input_layer = Input(shape=4096)
output = Dense(units=len(nfaces), activation='softmax')(input_layer)
clf = Model(input_layer, output)
clf.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
clf.summary()
accuracy = []

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

def build_fresh_classifier():
    input_layer = Input(shape=4096)
    output = Dense(units=len(nfaces), activation='softmax')(input_layer)
    model = Model(input_layer, output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

# EXPERIMENT 1: The "Control" (Standard Split)
print("\n--- RUNNING EXPERIMENT 1: STANDARD SPLIT (Control) ---")
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(dataset, y, test_size=0.4, shuffle=True, stratify=y)

y_train_std_enc = tf.keras.utils.to_categorical(y_train_std, num_classes=len(nfaces))
X_train_std_enc = encoder.predict(X_train_std/255)
X_test_std_enc = encoder.predict(X_test_std/255)

# Train Model A
clf_std = build_fresh_classifier()
hist_std = clf_std.fit(X_train_std_enc, y_train_std_enc, batch_size=64, epochs=40, verbose=0, validation_split=0.1)

y_pred_std = np.argmax(clf_std.predict(X_test_std_enc), axis=1)
acc_std = accuracy_score(y_test_std, y_pred_std)
print(f"RESULT: Standard Split Accuracy = {acc_std*100:.2f}%")


# EXPERIMENT 2: The "Gap" (Single Sample Split)
print("\n--- RUNNING EXPERIMENT 2: SINGLE SAMPLE (Experimental) ---")
X_train_sspp = []
y_train_sspp = []
X_test_sspp = []
y_test_sspp = []

current_index = 0
for count in nfaces:
    # 1. Take ONLY the FIRST image for Training
    X_train_sspp.append(dataset[current_index])
    y_train_sspp.append(y[current_index])

    # 2. All others go to Testing
    if count > 1:
        X_test_sspp.append(dataset[current_index + 1 : current_index + count])
        y_test_sspp.append(y[current_index + 1 : current_index + count])
    current_index += count

X_train_sspp = np.array(X_train_sspp)
y_train_sspp = np.array(y_train_sspp)
X_test_sspp = np.vstack(X_test_sspp)
y_test_sspp = np.vstack(y_test_sspp)

y_train_sspp_enc = tf.keras.utils.to_categorical(y_train_sspp, num_classes=len(nfaces))
X_train_sspp_enc = encoder.predict(X_train_sspp/255)
X_test_sspp_enc = encoder.predict(X_test_sspp/255)

# Train Model B
clf_sspp = build_fresh_classifier()
hist_sspp = clf_sspp.fit(X_train_sspp_enc, y_train_sspp_enc, batch_size=64, epochs=40, verbose=0, validation_split=0.1)

y_pred_sspp = np.argmax(clf_sspp.predict(X_test_sspp_enc), axis=1)
acc_sspp = accuracy_score(y_test_sspp, y_pred_sspp)
print(f"RESULT: Single Sample Accuracy = {acc_sspp*100:.2f}%")


# Visualization
labels = ['Standard (Many Images)', 'Single Sample (1 Image)']
accuracies = [acc_std * 100, acc_sspp * 100]

plt.figure(figsize=(8, 6))
bars = plt.bar(labels, accuracies, color=['green', 'red'])
plt.ylabel('Accuracy (%)')
plt.title('Impact of Data Scarcity on Convolutional Autoencoder')
plt.ylim(0, 100)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f"{yval:.1f}%", ha='center', fontweight='bold')

plt.show()

clf = clf_sspp
X_test = X_test_sspp
y_test = y_test_sspp

In [ ]:
import matplotlib.pyplot as plt

# Create subplots
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

# Plot Loss for Standard Split
ax[0].plot(hist_std.history['loss'], color='b', label="Training loss")
ax[0].plot(hist_std.history['val_loss'], color='r', label="Validation loss")
ax[0].set_ylabel("Crossentropy Loss")
ax[0].legend(loc='best', shadow=True)
ax[0].set_title("Experiment 1: Standard Split Training Performance")

# Plot Accuracy for Standard Split
ax[1].plot(hist_std.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist_std.history['val_accuracy'], color='r', label="Validation accuracy")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].legend(loc='best', shadow=True)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 1. Compute the confusion matrix for the Standard Split
conf_matrix_std = confusion_matrix(y_test_std, y_pred_std)

# 2. Setup plot sizing
plt.rcParams.update({'font.size': 8})
plt.rcParams['figure.figsize'] = [12, 12]

# 3. Display the matrix
disp = ConfusionMatrixDisplay(
    confusion_matrix=conf_matrix_std,
    display_labels=range(1, len(nfaces) + 1)
)

disp.plot(cmap=plt.cm.Greens, colorbar=False)
plt.title("Confusion Matrix: Standard Split (Regular Training)")
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10, 8))

# Plot Loss
ax[0].plot(hist_sspp.history['loss'], color='b', label="Training loss")
ax[0].plot(hist_sspp.history['val_loss'], color='r', label="Validation loss")
ax[0].set_ylabel("Crossentropy Loss")
ax[0].legend(loc='best', shadow=True)
ax[0].set_title("Single Sample Training Performance")

# Plot Accuracy
ax[1].plot(hist_sspp.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist_sspp.history['val_accuracy'], color='r',label="Validation accuracy")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].legend(loc='best', shadow=True)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Predicting people's names on the Single Sample test set...")
t0 = time()

X_test_enc = encoder.predict(X_test/255)

# Predict
y_pred = clf.predict(X_test_enc)
y_pred = np.argmax(y_pred, axis=1)

print("done in %0.3fs" % (time() - t0))

target_names = [f"Person{n}" for n in range(1,39)]

print(classification_report(y_test.ravel(), y_pred, target_names=target_names))

confusion_mtx = confusion_matrix(y_test.ravel(), y_pred)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

plt.rcParams.update({'font.size': 8})
plt.rcParams['figure.figsize'] = [12, 12]

# Display the matrix for the Single Sample experiment
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx, display_labels=range(1, 39))

disp.plot(cmap=plt.cm.Reds, colorbar=False)
plt.title("Confusion Matrix: Single Sample Per Person")
plt.show()

In [ ]:
# plot the result of the prediction on a portion of the test set
prediction_titles = [title(y_pred, y_test, target_names, i) for i in range(y_pred.shape[0])]
plot_gallery(X_test, prediction_titles, m, n, rgb=True)

# Reconstruction

In [ ]:
from sklearn.model_selection import train_test_split
from PIL import Image

TARGET_SIZE = (64, 64)

imgs = np.array(dataset)

processed = []

for i, im in enumerate(imgs):
    im_uint8 = im.astype("uint8")
    im_pil = Image.fromarray(im_uint8)
    im_gray = im_pil.convert("L").resize(TARGET_SIZE, Image.BILINEAR)
    arr = np.array(im_gray).astype("float32") / 255.0
    processed.append(arr)

processed = np.stack(processed)
processed = processed[..., np.newaxis]

print("Processed image shape:", processed.shape)

# Train/test split for reconstruction
x_train, x_test = train_test_split(processed, test_size=0.2, random_state=42, shuffle=True)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)

In [ ]:
from tensorflow.keras import layers, models

latent_dim = 128

# Encoder
encoder_input = layers.Input(shape=(64, 64, 1))
x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2,2), padding='same')(x)   # 32×32
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), padding='same')(x)   # 16×16
x = layers.Flatten()(x)
encoder_output = layers.Dense(latent_dim, activation='relu')(x)

encoder = models.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(16*16*64, activation='relu')(decoder_input)
x = layers.Reshape((16, 16, 64))(x)
x = layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu', padding='same')(x)  # 32×32
x = layers.Conv2DTranspose(32, (3,3), strides=2, activation='relu', padding='same')(x)  # 64×64
decoder_output = layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

decoder = models.Model(decoder_input, decoder_output, name="decoder")
decoder.summary()

# Autoencoder
autoencoder_output = decoder(encoder_output)
autoencoder = models.Model(encoder_input, autoencoder_output, name="autoencoder")
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

In [ ]:
history = autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=64,
    shuffle=True,
    validation_split=0.2
)

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(64,64), cmap='gray')
    plt.title("Original")
    plt.axis("off")

    # Reconstructed
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(64,64), cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")

plt.show()

In [ ]:
from skimage.metrics import structural_similarity as ssim

# Autoencoder prediction
decoded_imgs = autoencoder.predict(x_test)

mse_list = []
ssim_list = []

for i in range(len(x_test)):
    orig = x_test[i].reshape(64,64)
    recon = decoded_imgs[i].reshape(64,64)

    mse_value = np.mean((orig - recon)**2)
    ssim_value = ssim(orig, recon, data_range=1.0)

    mse_list.append(mse_value)
    ssim_list.append(ssim_value)

mean_mse = np.mean(mse_list)
mean_ssim = np.mean(ssim_list)

print("Reconstruction Accuracy")
print(f"Average MSE   : {mean_mse:.6f}")
print(f"Average SSIM  : {mean_ssim:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot the Mean Squared Error (MSE)
plt.plot(history.history['loss'], label='Training MSE', color='blue')
plt.plot(history.history['val_loss'], label='Validation MSE', color='red')

plt.title('Reconstruction Training Performance')
plt.ylabel('Mean Squared Error')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)

plt.show()